In [1]:
# !uv pip install kagglehub more_itertools torch transformers

In [2]:
import pandas as pd
import os

def is_submission():
    return os.getenv("KAGGLE_IS_COMPETITION_RERUN")

path = f"/kaggle/input/jigsaw-agile-community-rules/{'test.csv' if is_submission() else 'train.csv'}"

In [3]:
data = pd.read_csv(path)
data.head(1)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0


In [4]:
data['rule'][0]

'No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'

### Steps for initial solution

- Step - 1 Add a column for expanding the rule by describing the given rule and make it understandable along with the prompt.
- Step - 2 Each row updated with prompt with positive and negative examples. 
- Step - 3 Then finally setup system prompt by describing the nature of the tool and get the otuput as voilation or nonvoilation along with other information.
            

In [5]:
import kagglehub
import more_itertools
import pandas as pd
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [6]:
rule = data['rule'][0]

In [7]:
GEMMA_PATH = kagglehub.model_download("google/gemma-3/transformers/gemma-3-1b-it")
processor = AutoTokenizer.from_pretrained(GEMMA_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(GEMMA_PATH).to(device)

2025-10-03 17:08:21.042682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759511301.258300      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759511301.318543      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
### prompt for the rule exploration and expand the keywords.

SYS_PROMPT_RULE = """
You are an expert with understanding of community guidlines and related keywords.
For a given community rule provided, you go through its keywords and exapand each keyword in details. 

Output should only contain the expanded rules no explanatory text.
"""

USER_PROMPT_RULE = f"""
Expand below rule

{rule}

"""

# prompt_rule_text = (
#     SYS_PROMPT_RULE + "\n\n" +
#     USER_PROMPT_RULE
# )

messages = [
    {"role": "system", "content": SYS_PROMPT_RULE},
    {"role": "user", "content": USER_PROMPT_RULE}
]



prompt_rule_text = (
    f"<start_of_turn>system\n{SYS_PROMPT_RULE}<end_of_turn>\n"
    f"<start_of_turn>user\n{USER_PROMPT_RULE}<end_of_turn>\n"
    f"<start_of_turn>model\n"
)

rule_inputs = processor(prompt_rule_text, return_tensors="pt").to(device)

from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnString(StoppingCriteria):
    def __init__(self, tokenizer, stop_str):
        self.tokenizer = tokenizer
        self.stop_str = stop_str
        self.stop_ids = tokenizer.encode(stop_str, add_special_tokens=False)

    def __call__(self, input_ids, scores, **kwargs):
        if self.stop_ids == input_ids[0, -len(self.stop_ids):].tolist():
            return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnString(processor, "\n\n\n")])

# output_rules = model.generate(
#     **rule_inputs,
#     max_new_tokens=300,
#     do_sample=False,
#     repetition_penalty=1.2,
#     eos_token_id=processor.eos_token_id,
#     pad_token_id=processor.pad_token_id or processor.eos_token_id,
#     return_dict_in_generate=True
# )

# output_rules = model.generate(**rule_inputs, 
#                             max_new_tokens=300,
#                             temperature=0.5,
#                             top_p=1,
#                             repetition_penalty=1.2,
#                             eos_token_id=processor.eos_token_id,  # stop at EOS
#                             pad_token_id=processor.eos_token_id,
#                             return_dict_in_generate=True)

output_rules = model.generate(
    **rule_inputs,
    max_new_tokens=300,
    repetition_penalty=1.3,
    eos_token_id=processor.eos_token_id,
    pad_token_id=processor.eos_token_id,
    return_dict_in_generate=True
)

generated_tokens = output_rules.sequences[:, rule_inputs["input_ids"].shape[-1]:]
decoded_text = processor.batch_decode(generated_tokens, skip_special_tokens=True)
print(decoded_text[0].split("--")[0])

**Advertising:**  Promotion or commercialization of goods or services without permission from the community guidelines. This includes but is not limited to banner ads, sponsored posts, paid promotions (e.g., direct advertisements), affiliate marketing, contests/giveaways that require payment for entry, and any other form of soliciting revenue directly from users.


**Spam:** Unsolicited messages sent repeatedly or excessively, designed to deceive or mislead users into taking action against the platform’s terms of service. Examples include automated replies, repetitive requests, misleading offers, and mass email campaigns.



**Referral Links:** Any link shared by individuals attempting to promote another user's profile, product, or service within the community. These links must be genuine referrals and comply with all relevant policies regarding acceptable promotion methods.




**Unsolicited Content:** Messages or materials received outside of the intended distribution channels – such

In [13]:
import re
updated_rule = decoded_text[0].split("--")[0]

In [14]:
rule = updated_rule
target_comment = data['rule'][0]
positive_comment1 = data['positive_example_1'][0]
positive_comment2 = data['positive_example_2'][0]
negative_comment1 = data['negative_example_1'][0]
negative_comment2 = data['negative_example_2'][0]

In [17]:
SYSTEM_PROMPT_CLASSIFICATION = """

you are a community rule voilation inspector. 

if you are give the rule with context and target comment along with negative and positive comments you can decide if the target comment is in voilation of the specific rule.

you will use positive comment to understand how those comments voilates the rule and negative comments to understand why negative comments are not breaking the rules,

you are required to answer only one of the 2 values, 0 or 1. 0 denotes "not voilation" and 1 denotes "voilation".

output should be given as: 

{
    voilation: 1 or 0,
    confidence_score: <between 0 and 1, higher the better.>,
    justification: <reason why specific voilation value was choosen.>

}

"""


USER_PROMPT_CLASSIFICATION = f"""
Classify the below comment based on the rule given, 

comment = {target_comment}

below are the details related to the comment: 

Rule : {rule}
positive_comment1 = {positive_comment1}
positive_comment2 = {positive_comment2}
negative_comment1 = {negative_comment1}
negative_comment2 = {negative_comment2}

"""

prompt_classify_text = (
    f"<start_of_turn>system\n{SYSTEM_PROMPT_CLASSIFICATION}<end_of_turn>\n"
    f"<start_of_turn>user\n{USER_PROMPT_CLASSIFICATION}<end_of_turn>\n"
    f"<start_of_turn>model\n"
)

classifier_inputs = processor(prompt_classify_text, return_tensors="pt").to(device)



In [21]:
output_classify = model.generate(
    **classifier_inputs,
    max_new_tokens=100,
    repetition_penalty=1.3,
    eos_token_id=processor.eos_token_id,
    pad_token_id=processor.eos_token_id,
    return_dict_in_generate=True
)

generated_tokens = output_classify.sequences[:, classifier_inputs["input_ids"].shape[-1]:]
decoded_text = processor.batch_decode(generated_tokens, skip_special_tokens=True)
print(decoded_text[0].split("--")[0])

```json
{
    "voilation": 1,
    "confidence_score": 1.0,
    "justification": "The comment promotes explicit sexual activity ('double cee x bands eps') which violates the strict prohibition against promoting sexually suggestive content and solicitation of revenue."
}
```https://large-ocr.net/?text=%65No%20advertising:%20spam#section%3A%65No%20
